In [12]:
import pandas as pd
import numpy as np
week1_raw = pd.read_csv("week1_results.csv")
week2_raw = pd.read_csv("Week2_results.csv")
#week2_raw = pd.read_csv("week2_results.csv)
#week1_raw.head()



In [13]:
#function to combine picks and point values
def combine(df1):
    my_file = df1
    rows = my_file.shape[0]
    i = 2 
    while i < rows:
        for col in my_file.loc[:, my_file.columns != 'Game']:
            my_file[col][i+1] = float(my_file[col][i+1])
            my_file[col][i] = [my_file[col][i], my_file[col][i+1]]
        i = i+2
    return my_file

week1 = combine(week1_raw)
week2 = combine(week2_raw)

In [14]:
#drop unneeded rows
week1.drop([5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41], axis=0, inplace=True)
week2.drop([5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41], axis=0, inplace=True)
week2.tail()

#i = 3
#rows = week1.shape[1]
#while i < 6:
#    print(week1.iloc[])
#    i = i + 2

,Game,LAC at KC,CAR at NYG,NE at PIT,TB at NO,NYJ at CLE,MIA at BAL,WAS at DET,IND at JAX,ATL at LAR,SEA at SF,HOU at Den,ARI at LV,CIN at DAL,CHI at GB,TEN at BUF,MIN at PHI
32,taz7611,"[KC, 5.0]","[NYG, 8.0]","[PIT, 11.0]","[TB, 9.0]","[CLE, 3.0]","[BAL, 12.0]","[DET, 6.0]","[IND, 4.0]","[LAR, 15.0]","[SEA, 2.0]","[DEN, 14.0]","[ARI, 10.0]","[CIN, 13.0]","[CHI, 1.0]","[BUF, 16.0]","[MIN, 7.0]"
34,Team Boylan,"[LAC, 2.0]","[CAR, 8.0]","[PIT, 5.0]","[TB, 13.0]","[CLE, 9.0]","[BAL, 4.0]","[DET, 7.0]","[IND, 15.0]","[LAR, 11.0]","[SF, 6.0]","[DEN, 14.0]","[LV, 3.0]","[CIN, 12.0]","[GB, 10.0]","[BUF, 16.0]","[MIN, 1.0]"
36,UrBoiJZ,"[KC, 8.0]","[CAR, 12.0]","[PIT, 6.0]","[TB, 4.0]","[CLE, 15.0]","[BAL, 10.0]","[WAS, 11.0]","[IND, 1.0]","[LAR, 13.0]","[SEA, 9.0]","[HOU, 3.0]","[LV, 5.0]","[DAL, 7.0]","[CHI, 2.0]","[BUF, 16.0]","[MIN, 14.0]"
38,Wild Bill,"[KC, 13.0]","[NYG, 6.0]","[PIT, 14.0]","[TB, 15.0]","[CLE, 5.0]","[BAL, 12.0]","[DET, 4.0]","[IND, 11.0]","[LAR, 2.0]","[SF, 7.0]","[DEN, 8.0]","[LV, 10.0]","[CIN, 3.0]","[CHI, 1.0]","[BUF, 16.0]","[MIN, 9.0]"
40,Zapp,"[KC, 2.0]","[CAR, 7.0]","[NE, 9.0]","[TB, 4.0]","[CLE, 10.0]","[BAL, 3.0]","[DET, 8.0]","[IND, 11.0]","[LAR, 12.0]","[SF, 13.0]","[DEN, 14.0]","[LV, 6.0]","[CIN, 15.0]","[GB, 16.0]","[BUF, 5.0]","[PHI, 1.0]"


In [ ]:
#add spread for home team
#for col in week1.columns:
#    week1[col][1] = [week1[col][1], -week1[col][2][1]]


In [31]:
#function to calculate total points
def totals(df):
    i = 4
    df['Score'] = 0
    df['Game Count'] = 0
    while i < 41:
        for col in df.loc[:, ~df.columns.isin(['Game', 'Score', 'Game Count'])]:
            if df[col][0] == df[col][i][0]:
                df['Game Count'][i] = df['Game Count'][i] + 1
                df['Score'][i] = df['Score'][i] + df[col][i][1]
        i = i + 2
    return df 

In [32]:
week1_tot = totals(week1)
#week2 = totals(week2)

C:\Users\mzwartz\AppData\Local\Temp\ipykernel_21556\2909851566.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Game Count'][i] = df['Game Count'][i] + 1
C:\Users\mzwartz\AppData\Local\Temp\ipykernel_21556\2909851566.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Score'][i] = df['Score'][i] + df[col][i][1]


In [34]:
#week1.sort_values(week1['Score'])
week1[['Game', 'Score', 'Game Count']]
#week2[['Game', 'Score']]

,Game,Score,Game Count
0,Winner,0,0
1,Away Team,0,0
2,Home team,0,0
3,Spread,0,0
4,abfarias,52,7
6,Bob Z,72,11
8,Ellieisawesome1,61,8
10,jjapcon,60,8
12,Kathy Zwartz,97,13
14,Ken,67,9


In [44]:
#upset count

def upset_counter(df):
    upset_count = 0
    fav_count = 0
    tie_count = 0
    for col in week1.loc[:, ~week1.columns.isin(['Game', 'Score'])]:
        if week1[col][3] < 0 and week1[col][1] == week1[col][0]: #if home team is favorite and away team won
            upset_count = upset_count + 1
            #print('Home dog pulling it off: ', week1[col][1])
        elif week1[col][3] > 0 and week1[col][2][0] == week1[col][0]: #if away team is favorite and home team won
            upset_count = upset_count + 1
            #print('Road upset alert: ', week1[col][2][0])
        elif week1[col][0] == "TIE":
            tie_count = tie_count + 1
            #print("A friggen tie...  ", week1[col][2])
        else:
            fav_count = fav_count + 1
    
    return upset_count, fav_count, tie_count

In [45]:
upset_count1 = upset_counter(week1)[0]
fav_count1 = upset_counter(week1)[1]
tie_count1 = upset_counter(week1)[2]

upset_count2 = upset_counter(week2)[0]


print("There were ", upset_count1, " upsets in week 1.")
print("So far there have been ", upset_count2, " upsets in week 2.")


There were  6  upsets in week 1.
So far there have been  6  upsets in week 2.


In [38]:
#upset_score = 0
#for col in week1.columns:
    #if week1[col][1][1] == week1[0]:
    #    upset_score = upset_score + week1[col][1][1]
    #else if week1[0] == week1[col][1][1]
